In [2]:
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn import svm

In [48]:
df = pd.read_csv("dataset_eeg_nseg.csv")

In [49]:
df.drop(['Unnamed: 0'],axis=1, inplace=True)

In [5]:
df = df.set_index(['id_user','session','video'])

In [6]:
df

0         1         2         3         4  \
id_user session video                                                     
10      1       1      0.000056  0.000210  0.000578  0.001068  0.001386   
                2      0.000073  0.000045  0.000349  0.000667  0.000553   
                3      0.000053  0.000243  0.000697  0.000991  0.000822   
                4      0.000032  0.000114  0.000367  0.000646  0.000741   
                5      0.000037  0.000115  0.000549  0.000856  0.000560   
...                         ...       ...       ...       ...       ...   
9       3       20     0.000005  0.000028  0.000144  0.000136  0.000052   
                21     0.000012  0.000020  0.000093  0.000228  0.000360   
                22     0.000006  0.000026  0.000080  0.000192  0.000179   
                23     0.000035  0.000116  0.000271  0.000244  0.000255   
                24     0.000016  0.000144  0.000096  0.000285  0.000275   

                              5         6         7         8         9  ...  \
id_user session video                                                    ...   
10      1       1      0.001648  0.000803  0.000437  0.000751  0.000383  ...   
                2      0.000305  0.000502  0.000579  0.000626  0.000521  ...   
                3      0.000288  0.000170  0.000315  0.000519  0.000392  ...   
                4      0.000520  0.000351  0.000349  0.000443  0.000537  ...   
                5      0.000430  0.000418  0.000234  0.000129  0.000227  ...   
...                         ...       ...       ...       ...       ...  ...   
9       3       20     0.000212  0.000210  0.000047  0.000127  0.000165  ...   
                21     0.000446  0.000264  0.000091  0.000087  0.000094  ...   
                22     0.000068  0.000094  0.000052  0.000129  0.000117  ...   
                23     0.000180  0.000203  0.000097  0.000168  0.000173  ...   
                24     0.000113  0.000217  0.000480  0.000640  0.000708  ...   

                           5502      5503      5504      5505      5506  \
id_user session video                                                     
10      1       1      0.680894  0.677372  0.678587  0.680154  0.679000   
                2      0.681449  0.680814  0.681055  0.679858  0.679219   
                3      0.680556  0.679891  0.680421  0.680982  0.680714   
                4      0.681528  0.681170  0.682441  0.681751  0.681922   
                5      0.681270  0.679830  0.681256  0.682198  0.681864   
...                         ...       ...       ...       ...       ...   
9       3       20     0.683626  0.684026  0.683992  0.683872  0.683737   
                21     0.683333  0.683626  0.683485  0.683547  0.683689   
                22     0.683917  0.683562  0.683447  0.683692  0.683361   
                23     0.683632  0.683680  0.683738  0.683844  0.683406   
                24     0.683428  0.683518  0.683481  0.683462  0.683615   

                           5507      5508      5509      5510  emotion  
id_user session video                                                   
10      1       1      0.679539  0.679375  0.678674  0.680591        1  
                2      0.680615  0.676147  0.680797  0.679779        2  
                3      0.680317  0.680449  0.680310  0.679916        3  
                4      0.682117  0.682258  0.681055  0.681194        0  
                5      0.680023  0.680439  0.680000  0.679143        2  
...                         ...       ...       ...       ...      ...  
9       3       20     0.683521  0.683858  0.683820  0.683911        0  
                21     0.683664  0.683373  0.683693  0.684037        2  
                22     0.683313  0.683518  0.684135  0.684174        0  
                23     0.683448  0.683663  0.683562  0.683567        1  
                24     0.684293  0.684325  0.684048  0.683871        0  

[1080 rows x 5512 columns]

## Split train e test
### Subject Dependent
16 trials train e 8 test
2 testing group for labels 

In [2]:
def split_sub_dep(df):    
    df_sd = df.reset_index()
    # generiamo 8 video su cui eseguiamo i test
    test_videos = np.random.randint(1, 25, size=8)
    print(test_videos)
    X_testSD = df_sd[df_sd['video'].isin(test_videos)]
    print(X_testSD.emotion.unique()) #check se sono rappresentate tutte le emozioni
    X_testSD.set_index(['id_user','session','video'], inplace=True)
    y_testSD = X_testSD.emotion
    X_testSD = X_testSD.drop(['emotion'],axis=1)
    X_trainSD = df_sd[~df_sd['video'].isin(test_videos)]
    X_trainSD.set_index(['id_user','session','video'], inplace=True)
    y_trainSD = X_trainSD.emotion
    X_trainSD = X_trainSD.drop(['emotion'],axis=1)
    return X_trainSD,y_trainSD,X_testSD,y_testSD

In [3]:
X_trainSD,y_trainSD,X_testSD,y_testSD = split_sub_dep(df)

NameError: name 'df' is not defined

In [80]:
clfSD = GradientBoostingClassifier(n_estimators=10, learning_rate=0.5, max_depth=2,random_state=0).fit(X_trainSD, y_trainSD)

In [81]:
clfSD.score(X_testSD, y_testSD)

0.20833333333333334

In [82]:
clfSD.score(X_trainSD, y_trainSD)

0.8069444444444445

In [77]:
rfSD = RandomForestClassifier(n_estimators=10, max_depth=10,random_state=42).fit(X_trainSD, y_trainSD)

In [83]:
rfSD.score(X_testSD, y_testSD)

0.24722222222222223

In [84]:
rfSD.score(X_trainSD, y_trainSD)

0.9319444444444445

### Subject Indipendent
Leave One Out Cross Validation sugli utenti

In [23]:
def SubjectIndipendent_GBC(df):
    df_SI = df.reset_index()
    groups = df_SI['id_user']
    X_SI = df_SI.drop(['emotion','id_user','session','video'],axis=1)
    y_SI = df_SI.emotion
    return X_SI,y_SI,groups

In [86]:
X_SI,y_SI,groups = SubjectIndipendent_GBC(df)

In [87]:
df_SI = df.reset_index()
groups = df_SI['id_user']

In [88]:
groups

0       10
1       10
2       10
3       10
4       10
        ..
1075     9
1076     9
1077     9
1078     9
1079     9
Name: id_user, Length: 1080, dtype: int64

In [89]:
X_SI = df_SI.drop(['emotion','id_user','session','video'],axis=1)

In [90]:
X_SI

,0,1,2,3,4,5,6,7,8,9,...,5501,5502,5503,5504,5505,5506,5507,5508,5509,5510
0,0.000056,0.000210,0.000578,0.001068,0.001386,0.001648,0.000803,0.000437,0.000751,0.000383,...,0.681325,0.680894,0.677372,0.678587,0.680154,0.679000,0.679539,0.679375,0.678674,0.680591
1,0.000073,0.000045,0.000349,0.000667,0.000553,0.000305,0.000502,0.000579,0.000626,0.000521,...,0.680923,0.681449,0.680814,0.681055,0.679858,0.679219,0.680615,0.676147,0.680797,0.679779
2,0.000053,0.000243,0.000697,0.000991,0.000822,0.000288,0.000170,0.000315,0.000519,0.000392,...,0.680739,0.680556,0.679891,0.680421,0.680982,0.680714,0.680317,0.680449,0.680310,0.679916
3,0.000032,0.000114,0.000367,0.000646,0.000741,0.000520,0.000351,0.000349,0.000443,0.000537,...,0.681769,0.681528,0.681170,0.682441,0.681751,0.681922,0.682117,0.682258,0.681055,0.681194
4,0.000037,0.000115,0.000549,0.000856,0.000560,0.000430,0.000418,0.000234,0.000129,0.000227,...,0.682273,0.681270,0.679830,0.681256,0.682198,0.681864,0.680023,0.680439,0.680000,0.679143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1075,0.000005,0.000028,0.000144,0.000136,0.000052,0.000212,0.000210,0.000047,0.000127,0.000165,...,0.683606,0.683626,0.684026,0.683992,0.683872,0.683737,0.683521,0.683858,0.683820,0.683911
1076,0.000012,0.000020,0.000093,0.000228,0.000360,0.000446,0.000264,0.000091,0.000087,0.000094,...,0.683356,0.683333,0.683626,0.683485,0.683547,0.683689,0.683664,0.683373,0.683693,0.684037
1077,0.000006,0.000026,0.000080,0.000192,0.000179,0.000068,0.000094,0.000052,0.000129,0.000117,...,0.683923,0.683917,0.683562,0.683447,0.683692,0.683361,0.683313,0.683518,0.684135,0.684174
1078,0.000035,0.000116,0.000271,0.000244,0.000255,0.000180,0.000203,0.000097,0.000168,0.000173,...,0.683325,0.683632,0.683680,0.683738,0.683844,0.683406,0.683448,0.683663,0.683562,0.683567


In [91]:
y_SI = df_SI.emotion

In [92]:
y_SI

0       1
1       2
2       3
3       0
4       2
       ..
1075    0
1076    2
1077    0
1078    1
1079    0
Name: emotion, Length: 1080, dtype: int64

#### GradientBoostingClassifier

In [98]:
model = GradientBoostingClassifier(n_estimators=10, learning_rate=1, max_depth=5, random_state=0)

In [141]:
rfc = RandomForestClassifier(n_estimators=15, max_depth=10,random_state=22)

In [99]:
logo = LeaveOneGroupOut()

In [100]:
logo.get_n_splits(X_SI, y_SI, groups)

15

In [142]:
scores = cross_val_score(rfc, X_SI, y_SI, cv=logo, verbose=1 , groups = groups, n_jobs = -1)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:    1.5s finished


In [102]:
scores

array([0.23611111, 0.23611111, 0.27777778, 0.27777778, 0.19444444,
       0.19444444, 0.40277778, 0.20833333, 0.27777778, 0.34722222,
       0.31944444, 0.25      , 0.36111111, 0.34722222, 0.33333333])

In [103]:
scori = pd.DataFrame(scores)
scori.to_csv("scores.csv")

In [143]:
scores.mean()

0.30185185185185187

# PROVA INDIP

In [50]:
def split_for_session(df):
    df_list = []
    for i in range(1,4):
        df_sess = df.loc[df['session'] == i ]
        df_list.append(df_sess)
    return df_list

In [51]:
df_list = split_for_session(df)

In [52]:
df_list

[            0         1         2         3         4         5         6  \
 0    0.000056  0.000210  0.000578  0.001068  0.001386  0.001648  0.000803   
 1    0.000073  0.000045  0.000349  0.000667  0.000553  0.000305  0.000502   
 2    0.000053  0.000243  0.000697  0.000991  0.000822  0.000288  0.000170   
 3    0.000032  0.000114  0.000367  0.000646  0.000741  0.000520  0.000351   
 4    0.000037  0.000115  0.000549  0.000856  0.000560  0.000430  0.000418   
 ..        ...       ...       ...       ...       ...       ...       ...   
 355  0.000003  0.000022  0.000094  0.000726  0.001687  0.002499  0.001502   
 356  0.000042  0.000213  0.000395  0.000988  0.001144  0.000889  0.000822   
 357  0.000164  0.000267  0.000577  0.000950  0.000760  0.001177  0.001391   
 358  0.000027  0.000099  0.000463  0.000541  0.000250  0.000484  0.000387   
 359  0.000019  0.000064  0.000399  0.000595  0.000284  0.000455  0.000402   
 
             7         8         9  ...      5505      5506   

In [54]:
labels_df = []
groups = []
train_df = []
for df in df_list:
    labels_df.append(df["emotion"])
    groups.append(df["id_user"])
    train_df.append(df.drop(['emotion','id_user','session','video'],axis=1))

In [67]:
model = GradientBoostingClassifier(n_estimators=50, learning_rate=0.5, max_depth=5, random_state=0)

In [68]:
logo = LeaveOneGroupOut()

In [69]:
score = []
for i in range(0,3):
    scores = cross_val_score(model, train_df[i], labels_df[i] , cv=logo, verbose=1 , groups = groups[i], n_jobs = 5)
    score.append(scores)

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  15 out of  15 | elapsed:  8.3min finished
[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  15 out of  15 | elapsed:  8.4min finished
[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  15 out of  15 | elapsed:  8.3min finished


In [70]:
print(score[0].mean(),score[1].mean(),score[2].mean())

0.3694444444444445 0.36666666666666664 0.2972222222222223


### Subject-Biased Experiment
train_test_split

In [94]:
X_trainSB, X_testSB, y_trainSB, y_testSB = train_test_split(df.drop('emotion',axis=1), df.emotion, test_size=0.20, random_state=22)

In [95]:
X_testSB

,0,1,2,3,4,5,6,7,8,9,...,5504,5505,5506,5507,5508,5509,5510,id_user,session,video
902,1.655497e-05,0.000163,0.000662,0.001085,0.001105,0.001345,0.000904,0.000980,0.000970,0.000564,...,0.680337,0.679727,0.681680,0.682028,0.680669,0.680354,0.677878,2,3,15
1027,2.827807e-06,0.000063,0.000243,0.000103,0.000222,0.000271,0.000340,0.000469,0.000625,0.000405,...,0.684922,0.681160,0.680964,0.683143,0.683025,0.679897,0.676935,7,3,20
1054,2.366847e-04,0.000481,0.001083,0.001402,0.001490,0.000939,0.000439,0.000178,0.000277,0.000251,...,0.681810,0.681796,0.681786,0.682741,0.682450,0.681726,0.681240,8,3,23
921,7.745030e-07,0.000005,0.000004,0.000002,0.000017,0.000019,0.000013,0.000010,0.000007,0.000001,...,0.683676,0.684805,0.685673,0.687472,0.684892,0.682830,0.681930,3,3,10
877,2.593214e-05,0.000280,0.001144,0.001033,0.000358,0.000320,0.001033,0.000953,0.000406,0.000195,...,0.684246,0.684768,0.683733,0.683509,0.684585,0.684094,0.683711,1,3,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1006,9.059595e-05,0.000417,0.001989,0.002774,0.001586,0.000553,0.000447,0.000153,0.000223,0.000117,...,0.681923,0.682822,0.683701,0.682408,0.681915,0.680073,0.680024,6,3,23
1067,8.522036e-06,0.000069,0.000172,0.000190,0.000099,0.000079,0.000127,0.000143,0.000071,0.000056,...,0.683046,0.683490,0.683647,0.683715,0.683565,0.683596,0.683533,9,3,12
947,3.644635e-05,0.000463,0.001201,0.000978,0.001245,0.001143,0.000852,0.001477,0.001521,0.002319,...,0.683151,0.681917,0.684033,0.684328,0.684779,0.683353,0.679842,4,3,12
1025,6.618577e-05,0.000261,0.000423,0.000651,0.001353,0.000404,0.000949,0.000974,0.001042,0.000713,...,0.680253,0.678215,0.674903,0.680124,0.680059,0.682092,0.690817,7,3,18


#### GradientBoostingClassifier

In [96]:
clf = GradientBoostingClassifier(n_estimators=5, learning_rate=0.1, max_depth=3, random_state=42).fit(X_trainSB, y_trainSB)

In [97]:
clf.score(X_testSB, y_testSB)

0.9027777777777778

In [98]:
clf.score(X_trainSB, y_trainSB)

0.9756944444444444

In [82]:
rfc = RandomForestClassifier(n_estimators=10, max_depth=5,random_state=22).fit(X_trainSB, y_trainSB)

In [83]:
rfc.score(X_testSB, y_testSB)

0.3888888888888889

In [84]:
rfc.score(X_trainSB, y_trainSB)

0.8923611111111112

# GRADIENT BOOSTING ON ONE FEATURE
### PSD

In [6]:
df = pd.read_csv("dataset_eeg_psds.csv")
df.drop(['Unnamed: 0'],axis=1, inplace=True)
df = df.set_index(['id_user','session','video'])

In [167]:
df

0         1          2          3          4  \
id_user session video                                                        
10      1       1      0.952755  3.834360  10.721228  19.887183  25.824439   
                2      2.393555  1.443329  11.871719  22.774562  18.887957   
                3      1.489037  7.202926  20.856213  29.713532  24.614091   
                4      0.905984  3.543481  11.645386  20.604895  23.629156   
                5      1.370372  4.415421  21.411094  33.434142  21.845199   
...                         ...       ...        ...        ...        ...   
9       3       20     0.457711  3.349578  17.589614  16.559618   6.258637   
                21     0.644106  1.202051   5.968025  14.754987  23.375220   
                22     0.540826  2.809283   8.986773  21.860166  20.385859   
                23     2.890024  9.788639  22.987859  20.691225  21.647931   
                24     0.471703  4.913602   3.242203   9.858747   9.511241   

                               5          6          7          8          9  \
id_user session video                                                          
10      1       1      30.715281  14.928749   8.090955  13.951526   7.070147   
                2      10.355057  17.133717  19.765291  21.365868  17.781661   
                3       8.567110   5.005052   9.369096  15.508441  11.682358   
                4      16.567218  11.142225  11.079863  14.100062  17.095886   
                5      16.761699  16.284967   9.088370   4.955161   8.809823   
...                          ...        ...        ...        ...        ...   
9       3       20     25.875669  25.595647   5.699863  15.477017  20.101218   
                21     28.982387  17.146680   5.808435   5.575698   6.014293   
                22      7.608486  10.691958   5.785502  14.618375  13.320158   
                23     15.249477  17.169880   8.146258  14.209363  14.635962   
                24      3.831531   7.480618  16.649001  22.263723  24.623149   

                       ...      2997      2998      2999      3000      3001  \
id_user session video  ...                                                     
10      1       1      ... -0.086706 -0.086706 -0.086706 -0.086706 -0.086706   
                2      ... -0.095146 -0.095146 -0.095146 -0.095146 -0.095146   
                3      ... -0.092203 -0.092203 -0.092203 -0.092203 -0.092203   
                4      ... -0.107096 -0.107096 -0.107096 -0.107096 -0.107096   
                5      ... -0.078886 -0.078886 -0.078886 -0.078886 -0.078886   
...                    ...       ...       ...       ...       ...       ...   
9       3       20     ... -0.100242 -0.100242 -0.100242 -0.100242 -0.100242   
                21     ... -0.109461 -0.109461 -0.109461 -0.109461 -0.109461   
                22     ... -0.134824 -0.134824 -0.134824 -0.134824 -0.134824   
                23     ... -0.099847 -0.099847 -0.099847 -0.099847 -0.099847   
                24     ... -0.100633 -0.100633 -0.100633 -0.100633 -0.100633   

                           3002      3003      3004      3005  emotion  
id_user session video                                                   
10      1       1     -0.086706 -0.086706 -0.086706 -0.086706        1  
                2     -0.095146 -0.095146 -0.095146 -0.095146        2  
                3     -0.092203 -0.092203 -0.092203 -0.092203        3  
                4     -0.107096 -0.107096 -0.107096 -0.107096        0  
                5     -0.078886 -0.078886 -0.078886 -0.078886        2  
...                         ...       ...       ...       ...      ...  
9       3       20    -0.100242 -0.100242 -0.100242 -0.100242        0  
                21    -0.109461 -0.109461 -0.109461 -0.109461        2  
                22    -0.134824 -0.134824 -0.134824 -0.134824        0  
                23    -0.099847 -0.099847 -0.099847 -0.099847        1  
                24    -0.100633 -0.100633 -0.100633 -0.10

## DEPENDENCY

In [7]:
X_trainSD,y_trainSD,X_testSD,y_testSD = split_sub_dep(df)

[18  2 16  5  3 12 20 22]
[2 3 1 0]


In [224]:
clfSD = GradientBoostingClassifier(n_estimators=20, learning_rate=0.2, max_depth=2,random_state=22).fit(X_trainSD, y_trainSD)

In [225]:
clfSD.score(X_trainSD, y_trainSD)

0.8915032679738563

In [226]:
clfSD.score(X_testSD, y_testSD)

0.25396825396825395

In [186]:
y_pred = clfSD.predict(X_testSD)

In [187]:
print(classification_report(y_testSD,y_pred))

              precision    recall  f1-score   support

           0       0.41      0.20      0.27       120
           1       0.18      0.13      0.15        75
           2       0.16      0.67      0.26        30
           3       0.33      0.29      0.31        90

    accuracy                           0.25       315
   macro avg       0.27      0.32      0.25       315
weighted avg       0.31      0.25      0.25       315



In [262]:
rfc = RandomForestClassifier(n_estimators=30, max_depth=6,random_state=42).fit(X_trainSD, y_trainSD)

In [263]:
rfc.score(X_trainSD, y_trainSD)

0.8061728395061728

In [264]:
rfc.score(X_testSD, y_testSD)

0.24074074074074073

## INDIPENDENCY

In [265]:
X_SI,y_SI,groups = SubjectIndipendent_GBC(df)

In [46]:
model = GradientBoostingClassifier(n_estimators=20, learning_rate=1, max_depth=15, random_state=0)
logo = LeaveOneGroupOut()
logo.get_n_splits(X_SI, y_SI, groups)
scores = cross_val_score(model, X_SI, y_SI, cv=logo, verbose=1 , groups = groups, n_jobs = -1)
print(scores.mean())

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:  5.6min finished


0.31851851851851853


In [266]:
model = RandomForestClassifier(n_estimators=30, max_depth=6,random_state=42)
logo = LeaveOneGroupOut()
logo.get_n_splits(X_SI, y_SI, groups)
scores = cross_val_score(model, X_SI, y_SI, cv=logo, verbose=1 , groups = groups, n_jobs = -1)
print(scores.mean())

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:    3.1s finished


0.32500000000000007


## BIAS

In [275]:
X_trainSB, X_testSB, y_trainSB, y_testSB = train_test_split(df.drop('emotion',axis=1), df.emotion, test_size=0.20, random_state=42)

In [280]:
clf = GradientBoostingClassifier(n_estimators=10, learning_rate=0.1, max_depth=3, random_state=0).fit(X_trainSB, y_trainSB)

In [281]:
clf.score(X_testSB, y_testSB)

0.28703703703703703

In [282]:
clf.score(X_trainSB, y_trainSB)

0.7604166666666666

In [270]:
rfc = RandomForestClassifier(n_estimators=30, max_depth=6,random_state=42).fit(X_trainSB, y_trainSB)

In [271]:
rfc.score(X_testSB,y_testSB)

0.3472222222222222

In [278]:
rfc.score(X_trainSB,y_trainSB)

0.8067129629629629

# GRADIENT BOOSTING ON ONE FEATURE
### DE

In [39]:
df = pd.read_csv("dataset_eeg_des.csv")
df.drop(['Unnamed: 0'],axis=1, inplace=True)
df = df.set_index(['id_user','session','video'])

In [284]:
df

0         1         2         3         4  \
id_user session video                                                     
10      1       1      2.135436  2.887686  3.465504  3.810436  3.955764   
                2      2.549241  2.254038  3.489210  3.876865  3.767604   
                3      2.436902  3.456774  4.163457  4.401829  4.272075   
                4      1.841611  2.513253  3.124816  3.421180  3.492676   
                5      2.320207  2.961560  3.857012  4.111162  3.868319   
...                         ...       ...       ...       ...       ...   
9       3       20     1.398441  2.604782  3.687751  3.648057  3.009953   
                21     1.571485  1.894192  2.786308  3.305627  3.571761   
                22     1.270886  2.200624  2.915375  3.471395  3.427353   
                23     2.621798  3.373337  3.905828  3.839850  3.868095   
                24     1.494128  3.076038  2.783606  3.572978  3.547337   

                              5         6         7         8         9  ...  \
id_user session video                                                    ...   
10      1       1      4.053275  3.646478  3.300221  3.606062  3.223825  ...   
                2      3.407996  3.706311  3.790671  3.835916  3.726078  ...   
                3      3.567253  3.214838  3.632954  3.966298  3.774067  ...   
                4      3.306529  3.099885  3.100102  3.221046  3.320871  ...   
                5      3.717754  3.698447  3.367067  3.021535  3.351307  ...   
...                         ...       ...       ...       ...       ...  ...   
9       3       20     3.942856  3.935395  2.950334  3.604765  3.776026  ...   
                21     3.696536  3.393062  2.770488  2.746998  2.790538  ...   
                22     2.811443  3.023582  2.642579  3.219103  3.161054  ...   
                23     3.649109  3.723545  3.259310  3.605300  3.623675  ...   
                24     2.896757  3.371674  3.948618  4.159415  4.232633  ...   

                            492       493       494       495       496  \
id_user session video                                                     
10      1       1     -1.011859 -1.020659 -0.602725 -0.584323 -0.602491   
                2     -0.654046 -0.287855 -0.064083 -0.382718 -0.854156   
                3     -0.102593 -0.080281 -0.138853  0.001005 -0.083322   
                4     -0.990079 -0.927056 -0.745584 -0.978420 -1.031638   
                5     -0.247413 -0.758625 -1.643860 -1.252678 -0.605932   
...                         ...       ...       ...       ...       ...   
9       3       20    -0.389243 -0.347923 -0.460666 -0.483391 -0.797307   
                21    -0.804578 -1.045797 -1.104529 -0.949304 -0.719607   
                22    -0.531922 -0.662631 -0.867511 -1.125926 -0.903615   
                23    -0.544234 -0.343033 -0.551949 -0.976702 -1.112032   
                24    -0.602491  0.003543  0.088560 -0.193022 -0.742112   

                            497       498       499       500  emotion  
id_user session video                                                   
10      1       1     -0.579400 -1.200410 -0.910379 -1.369580        1  
                2     -0.538963 -0.978321 -0.520367 -1.108300        2  
                3     -0.067288 -0.063967  0.042125 -0.123110        3  
                4     -0.682465 -0.665910 -1.033460 -1.373817        0  
                5     -0.750933 -1.202377 -0.686867 -1.160245        2  
...                         ...       ...       ...       ...      ...  
9       3       20    -0.593601 -0.401147 -0.498099 -0.751132        0  
                21    -0.668664 -0.650504 -0.838495 -1.361561        2  
                22    -1.054831 -0.916811 -1.142544 -1.251611        0  
                23    -0.834240 -0.739685 -0.623760 -1.128035        1  
                24    -1.026718 -0.977749 -0.960097 -1.363363        0  

[1080 rows x 502 columns]

## DEPENDENCY

In [40]:
X_trainSD,y_trainSD,X_testSD,y_testSD = split_sub_dep(df)

[15 19 24 16 24  1 20 16]
[1 2 3 0]


In [41]:
clfSD = GradientBoostingClassifier(n_estimators=20, learning_rate=0.2, max_depth=2,random_state=10).fit(X_trainSD, y_trainSD)

In [42]:
clfSD.score(X_trainSD, y_trainSD)

0.8358024691358025

In [43]:
clfSD.score(X_testSD, y_testSD)

0.2740740740740741

In [44]:
rfc = RandomForestClassifier(n_estimators=30, max_depth=5,random_state=42).fit(X_trainSD, y_trainSD)

In [45]:
rfc.score(X_trainSD, y_trainSD)

0.8407407407407408

In [46]:
rfc.score(X_testSD, y_testSD)

0.25555555555555554

In [47]:
y_pred = clfSD.predict(X_testSD)

In [48]:
print(classification_report(y_testSD,y_pred))

              precision    recall  f1-score   support

           0       0.37      0.29      0.33        75
           1       0.23      0.19      0.21        75
           2       0.09      0.30      0.14        30
           3       0.54      0.32      0.40        90

    accuracy                           0.27       270
   macro avg       0.31      0.28      0.27       270
weighted avg       0.36      0.27      0.30       270



## INDIPENDENCY

In [317]:
X_SI,y_SI,groups = SubjectIndipendent_GBC(df)

In [320]:
model = GradientBoostingClassifier(n_estimators=20, learning_rate=0.1, max_depth=15, random_state=0)
logo = LeaveOneGroupOut()
logo.get_n_splits(X_SI, y_SI, groups)
scores = cross_val_score(model, X_SI, y_SI, cv=logo, verbose=1 , groups = groups, n_jobs = -1)
print(scores.mean())

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.


0.274074074074074


[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:  1.5min finished


In [321]:
model = RandomForestClassifier(n_estimators=30, max_depth=5,random_state=42)
logo = LeaveOneGroupOut()
logo.get_n_splits(X_SI, y_SI, groups)
scores = cross_val_score(model, X_SI, y_SI, cv=logo, verbose=1 , groups = groups, n_jobs = -1)
print(scores.mean())

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.


0.34444444444444444


[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:    0.6s finished


## BIAS

In [322]:
X_trainSB, X_testSB, y_trainSB, y_testSB = train_test_split(df.drop('emotion',axis=1), df.emotion, test_size=0.20, random_state=42)

In [329]:
clf = GradientBoostingClassifier(n_estimators=10, learning_rate=0.1, max_depth=3, random_state=0).fit(X_trainSB, y_trainSB)

In [330]:
clf.score(X_testSB, y_testSB)

0.32407407407407407

In [331]:
clf.score(X_trainSB, y_trainSB)

0.7662037037037037

In [332]:
rfc = RandomForestClassifier(n_estimators=30, max_depth=5,random_state=42).fit(X_trainSB,y_trainSB)

In [333]:
rfc.score(X_testSB,y_testSB)

0.33796296296296297

In [335]:
rfc.score(X_trainSB,y_trainSB)

0.8182870370370371

# GRADIENT BOOSTING ON ONE FEATURE
### ASM

In [2]:
df = pd.read_csv("dataset_eeg_asms.csv")
df.drop(['Unnamed: 0'],axis=1, inplace=True)
df = df.set_index(['id_user','session','video'])

In [3]:
df

0         1         2         3         4  \
id_user session video                                                     
10      1       1     -0.507886 -0.614872 -1.641967 -0.809440 -0.254521   
                2     -1.526726 -0.019622 -0.623337 -0.318292 -0.963419   
                3     -1.712901 -0.626136 -0.681488 -1.267282 -0.191965   
                4     -0.326937 -1.115830 -1.166505 -1.217275 -1.309265   
                5     -2.620650 -0.919027 -0.869191 -0.793411 -0.794974   
...                         ...       ...       ...       ...       ...   
9       3       20    -0.363106  0.264170  0.455100  0.413153  0.311342   
                21     0.845171  0.200099 -0.350465  0.690646  0.469665   
                22     0.101389  0.227249 -0.183157 -0.156111  0.094033   
                23     0.296960  0.046466  0.158650  0.339566  0.350600   
                24     0.984481 -2.887236 -5.109027 -1.020892 -1.191568   

                              5         6         7         8         9  ...  \
id_user session video                                                    ...   
10      1       1     -0.153257 -0.177033  0.468183  0.504833  0.720677  ...   
                2     -0.693255 -0.207848  0.014371  0.310071  0.304314  ...   
                3      0.529379 -0.171602 -1.190944 -0.759927  0.229540  ...   
                4     -0.652792 -0.228270 -1.581202  0.183983  0.267916  ...   
                5     -1.004569 -0.218292 -0.296775  0.353089 -0.760424  ...   
...                         ...       ...       ...       ...       ...  ...   
9       3       20     0.099790  0.290642 -0.595710 -0.430711  0.250864  ...   
                21     0.108025 -0.221799  0.112040  0.322455 -0.118011  ...   
                22     0.344920 -0.105058 -0.497562 -0.150350 -0.314806  ...   
                23     0.487405  0.065113 -0.011989  0.204698  0.287173  ...   
                24    -0.455939  1.777643  1.176339  0.583990  0.294424  ...   

                            492       493       494       495       496  \
id_user session video                                                     
10      1       1     -0.603754 -3.936397 -2.786970 -1.303838 -2.395775   
                2     -0.041316 -0.535183 -0.348228 -1.279361 -1.776960   
                3     -1.006256 -1.668214 -1.141022 -0.581752 -0.849187   
                4     -0.820992 -1.289149  0.374519 -0.529157 -0.304496   
                5     -0.323980 -1.570919 -0.336004  0.479314  0.190322   
...                         ...       ...       ...       ...       ...   
9       3       20    -0.236637  0.839615  0.747818  0.426037  0.061812   
                21    -1.719179 -0.353402 -1.006594 -0.720096 -0.058573   
                22     0.837657 -0.709266 -1.209290 -0.144284 -1.580553   
                23    -0.221597 -0.028114  0.207154  0.636081 -1.132681   
                24    -1.218383 -0.846308 -0.999152 -1.075692 -0.446103   

                            497       498       499       500  emotion  
id_user session video                                                   
10      1       1     -1.886432 -2.041642 -2.704514 -0.891223        1  
                2     -0.690190 -4.166999 -0.548480 -1.340570        2  
                3     -1.244093 -1.113199 -1.251479 -1.643447        3  
                4     -0.049593  0.135184 -1.440402 -1.258214        0  
                5     -1.403735 -1.043561 -1.423911 -2.165414        2  
...                         ...       ...       ...       ...      ...  
9       3       20    -0.519885  0.388118  0.286195  0.531003        0  
                21    -0.172256 -1.529113 -0.037419  1.565252        2  
                22    -1.788901 -0.900419  1.783097  1.951517        0  
                23    -0.960791 -0.096051 -0.502811 -0.483703        1  
                24     2.355415  2.488038  1.343486  0.613903        0  

[1080 rows x 502 columns]

## DEPENDENCY

In [7]:
X_trainSD,y_trainSD,X_testSD,y_testSD = split_sub_dep(df)

[13 16  6  9  2  6 11 13]
[2 0 1 3]


In [36]:
clfSD = GradientBoostingClassifier(n_estimators=25, learning_rate=0.2, max_depth=2,random_state=0).fit(X_trainSD, y_trainSD)

In [37]:
clfSD.score(X_trainSD, y_trainSD)

0.8864197530864197

In [38]:
clfSD.score(X_testSD, y_testSD)

0.2222222222222222

In [39]:
rfc = RandomForestClassifier(n_estimators=30, max_depth=7,random_state=42).fit(X_trainSD,y_trainSD)

In [40]:
rfc.score(X_testSD,y_testSD)

0.25925925925925924

In [41]:
rfc.score(X_trainSD,y_trainSD)

0.9506172839506173

In [21]:
y_pred = clfSD.predict(X_testSD)

In [ ]:
print(classification_report(y_testSD,y_pred))

## INDIPENDENCY

In [42]:
X_SI,y_SI,groups = SubjectIndipendent_GBC(df)

In [45]:
model = GradientBoostingClassifier(n_estimators=20, learning_rate=0.1, max_depth=7, random_state=0)
logo = LeaveOneGroupOut()
logo.get_n_splits(X_SI, y_SI, groups)
scores = cross_val_score(model, X_SI, y_SI, cv=logo, verbose=1 , groups = groups, n_jobs = -1)
print(scores.mean())

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


0.275


[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:  1.9min finished


In [46]:
model = RandomForestClassifier(n_estimators=30, max_depth=7,random_state=42)
logo = LeaveOneGroupOut()
logo.get_n_splits(X_SI, y_SI, groups)
scores = cross_val_score(model, X_SI, y_SI, cv=logo, verbose=1 , groups = groups, n_jobs = -1)
print(scores.mean())

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


0.2935185185185185


[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:    1.4s finished


## BIAS

In [48]:
X_trainSB, X_testSB, y_trainSB, y_testSB = train_test_split(df.drop('emotion',axis=1), df.emotion, test_size=0.20, random_state=42)

In [65]:
clf = GradientBoostingClassifier(n_estimators=20, learning_rate=0.1, max_depth=3, random_state=0).fit(X_trainSB, y_trainSB)

In [66]:
clf.score(X_testSB, y_testSB)

0.2962962962962963

In [67]:
clf.score(X_trainSB,y_trainSB)

0.8912037037037037

In [71]:
rfc = RandomForestClassifier(n_estimators=30, max_depth=7,random_state=42).fit(X_trainSB,y_trainSB)

In [72]:
rfc.score(X_trainSB,y_trainSB)

0.9641203703703703

In [73]:
rfc.score(X_testSB,y_testSB)

0.3101851851851852

# GRADIENT BOOSTING ON ONE FEATURE
### DASM

In [74]:
df = pd.read_csv("dataset_eeg_dasms.csv")
df.drop(['Unnamed: 0'],axis=1, inplace=True)
df = df.set_index(['id_user','session','video'])

In [75]:
df

0             1             2             3  \
id_user session video                                                           
10      1       1      12650.603507  12648.390584  12627.183801  12644.366178   
                2      23159.311016  23228.675318  23200.891697  23214.945153   
                3      20310.108415  20344.375682  20342.634717  20324.171070   
                4      21714.090429  21693.944995  21692.633774  21691.334161   
                5      26387.909596  26468.273255  26470.628686  26474.199258   
...                             ...           ...           ...           ...   
9       3       20     83232.652198  83262.416482  83271.476343  83269.486399   
                21     44377.498193  44368.071520  44360.024699  44375.239472   
                22     77909.122398  77912.585034  77901.294744  77902.037879   
                23     57872.923314  57867.404649  57869.875955  57873.861769   
                24     23737.507729  23703.337772  23683.724240  23719.810333   

                                  4             5             6             7  \
id_user session video                                                           
10      1       1      12655.821677  12657.912527  12657.423942  12670.734588   
                2      23185.253969  23197.676893  23220.022194  23230.254935   
                3      20358.065116  20380.788349  20358.704419  20326.576719   
                4      21688.980463  21705.762150  21716.607324  21682.026333   
                5      26474.122085  26464.252362  26501.362420  26497.658895   
...                             ...           ...           ...           ...   
9       3       20     83264.655727  83254.617406  83263.672846  83221.614911   
                21     44372.010338  44366.725660  44361.905632  44366.783955   
                22     77908.918529  77915.819825  77903.441838  77892.644420   
                23     57874.105052  57877.119209  57867.815270  57866.116360   
                24     23718.304043  23724.796173  23744.506479  23739.200124   

                                  8             9  ...           1494  \
id_user session video                              ...                  
10      1       1      12671.490607  12675.944154  ...   18690.330810   
                2      23243.865618  23243.604063  ...   34293.059359   
                3      20340.163577  20371.347796  ...   30060.106190   
                4      21727.137159  21729.282150  ...   32035.791191   
                5      26528.326749  26475.762155  ...   39129.674113   
...                             ...           ...  ...            ...   
9       3       20     83229.445640  83261.786274  ...  122478.488712   
                21     44369.859135  44363.422629  ...   65272.166365   
                22     77902.196130  77897.672069  ...  114614.792502   
                23     57870.890395  57872.707613  ...   85137.511743   
                24     23733.973069  23731.417918  ...   34909.847325   

                                1495           1496           1497  \
id_user session video                                                
10      1       1       18690.888346   18690.699299   18690.448836   
                2       34293.244473   34293.176953   34293.509391   
                3       30060.274793   30060.127199   30059.981530   
                4       32035.900896   32035.502696   32035.721090   
                5       39130.153228   39129.623449   39129.347652   
...                              ...            ...            ...   
9       3       20     122478.000338  122478.037504  122478.177637   
                21      65271.981952   65272.082634   65272.035110   
                22     114615.197337  114615.328005  114615.032510   
                23      85137.471387   85137.422563   85137.324035   
                24      34909.817159   34909.830729   34909.836524   

                                1498           1499           150

## DEPENDENCY

In [81]:
X_trainSD,y_trainSD,X_testSD,y_testSD = split_sub_dep(df)

[21 16 15 22 19 23 21 21]
[2 3 0 1]


In [104]:
clfSD = GradientBoostingClassifier(n_estimators=20, learning_rate=0.3, max_depth=4,random_state=0).fit(X_trainSD, y_trainSD)

In [105]:
clfSD.score(X_testSD, y_testSD)

0.2111111111111111

In [106]:
clfSD.score(X_trainSD,y_trainSD)

0.8074074074074075

In [101]:
rfc = RandomForestClassifier(n_estimators=40, max_depth=8,random_state=42).fit(X_trainSD,y_trainSD)

In [102]:
rfc.score(X_trainSD,y_trainSD)

0.7049382716049383

In [103]:
rfc.score(X_testSD,y_testSD)

0.24074074074074073

In [ ]:
y_pred = clfSD.predict(X_testSD)

In [ ]:
print(classification_report(y_testSD,y_pred))

## INDIPENDENCY

In [25]:
X_SI,y_SI,groups = SubjectIndipendent_GBC(df)

In [108]:
model = GradientBoostingClassifier(n_estimators=20, learning_rate=0.1, max_depth=5, random_state=0)
logo = LeaveOneGroupOut()
logo.get_n_splits(X_SI, y_SI, groups)
scores = cross_val_score(model, X_SI, y_SI, cv=logo, verbose=1 , groups = groups, n_jobs = -1)
print(scores.mean())

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:  4.3min finished


0.2537037037037037


In [26]:
model = RandomForestClassifier(n_estimators=20, max_depth=15, random_state=0)
logo = LeaveOneGroupOut()
logo.get_n_splits(X_SI, y_SI, groups)
scores = cross_val_score(model, X_SI, y_SI, cv=logo, verbose=1 , groups = groups, n_jobs = -1)
print(scores.mean())

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:    6.8s finished


0.31296296296296305


## BIAS

In [27]:
X_trainSB, X_testSB, y_trainSB, y_testSB = train_test_split(df.drop('emotion',axis=1), df.emotion, test_size=0.20, random_state=42)

In [29]:
clf = GradientBoostingClassifier(n_estimators=20, learning_rate=0.1, max_depth=3, random_state=0).fit(X_trainSB, y_trainSB)

In [30]:
clf.score(X_testSB, y_testSB)

0.3101851851851852

In [31]:
clf.score(X_trainSB, y_trainSB)

0.8518518518518519

In [36]:
rfc = RandomForestClassifier(n_estimators=20, max_depth=5, random_state=0).fit(X_trainSB,y_trainSB)

In [37]:
rfc.score(X_testSB,y_testSB)

0.3287037037037037

In [38]:
rfc.score(X_trainSB,y_trainSB)

0.6979166666666666